# Timer

In [1]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']#, 'Trend']
indicator_list = pattern
indicator_list_higher = []#['Trend']

work_timeframe = '1h'
higher_timeframe = '4h'
opt_limit = 100000
load = False 

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [35]
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod': [48], # np.arange(2, 102, 2), 
                          'low_bound': [0]
                        }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load, 'ml')

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


  0%|          | 0/1 [00:00<?, ?it/s]

Timeframe is 1h/4h, trade type is buy
Number of combinations is 1


100%|██████████| 1/1 [01:33<00:00, 93.21s/it]


# Check local statistics

In [2]:
# rsi + stoch
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,STOCH_RSI,14,35,9,7,3,25,1612,0.779779,-3.571667,-354.995983,-5757.526667,0.8357,0.7815,0.7911,0.7685,0.7299,0.7312,0.7249,0.7054,0.6866,0.6849,0.699,0.7072,0.7239,0.7494,0.7628,0.7634,0.7804,0.8016,0.8192,0.8446,0.8801,0.9037,0.9146,0.9251,-0.32,-0.65,-0.97,-1.3,-1.62,-1.96,-2.29,-2.62,-2.96,-3.29,-3.63,-3.95,-4.27,-4.53,-4.72,-4.88,-5.03,-5.15,-5.22,-5.27,-5.28,-5.28,-5.27,-5.26


In [3]:
# trend
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
7,STOCH_RSI_Trend,14,20,48,0.2,0.8,0.8,0.5,9,7,3,25,48,0,186,1.848063,3.287917,157.739625,611.552500,1.3368,1.3714,1.2615,1.2589,1.2331,1.2900,1.3682,1.3992,1.5507,1.6442,1.7687,1.9843,2.1608,2.1477,2.1703,2.2133,2.2364,2.2175,2.2365,2.2436,2.2777,2.2865,2.3194,2.3768,0.32,0.64,0.95,1.26,1.55,1.85,2.15,2.41,2.69,2.96,3.23,3.51,3.76,3.99,4.19,4.37,4.54,4.68,4.82,4.92,4.98,5.03,5.04,5.07
3,STOCH_RSI_Trend,14,15,48,0.2,0.8,0.8,0.5,9,7,3,25,48,0,57,1.636133,3.223750,36.259600,183.753750,1.5764,1.2524,1.1066,1.1051,1.0077,1.0426,1.1779,1.2077,1.2636,1.2917,1.4321,1.6950,1.9504,1.9422,1.9339,2.0608,2.0433,2.0228,2.0429,2.0415,2.0341,1.9978,2.0072,2.0315,0.32,0.63,0.93,1.23,1.51,1.79,2.08,2.36,2.64,2.91,3.19,3.46,3.73,3.95,4.15,4.33,4.49,4.62,4.73,4.82,4.87,4.88,4.87,4.88
6,STOCH_RSI_Trend,14,20,48,0.2,0.8,0.8,0.5,9,7,3,20,48,0,174,1.817696,3.182500,142.279075,553.755000,1.3464,1.3579,1.2337,1.2285,1.1920,1.2499,1.3292,1.3670,1.5292,1.6311,1.7266,1.9486,2.1149,2.1041,2.1224,2.1679,2.1992,2.1830,2.2078,2.2156,2.2519,2.2644,2.2986,2.3548,0.31,0.61,0.91,1.19,1.48,1.76,2.06,2.32,2.59,2.86,3.12,3.40,3.64,3.87,4.06,4.24,4.40,4.54,4.67,4.77,4.84,4.89,4.91,4.94
2,STOCH_RSI_Trend,14,15,48,0.2,0.8,0.8,0.5,9,7,3,20,48,0,56,1.573925,3.181250,32.139800,178.150000,1.5031,1.1387,1.0140,1.0197,0.9329,0.9726,1.1112,1.1420,1.1993,1.2289,1.3716,1.6364,1.8940,1.8878,1.8806,2.0084,1.9918,1.9730,1.9942,1.9931,1.9859,1.9504,1.9600,1.9846,0.31,0.62,0.92,1.21,1.49,1.77,2.05,2.34,2.61,2.88,3.15,3.42,3.68,3.90,4.10,4.28,4.43,4.56,4.67,4.76,4.80,4.81,4.79,4.80
1,STOCH_RSI_Trend,14,15,48,0.2,0.8,0.8,0.5,9,7,3,15,48,0,47,1.666929,3.137083,31.345671,147.442917,1.8162,1.2823,1.0696,1.0712,0.9551,1.0111,1.1798,1.2343,1.3050,1.3351,1.4707,1.7726,2.0679,2.0342,1.9983,2.1243,2.1011,2.0435,2.0437,2.0420,2.0330,1.9951,2.0057,2.0145,0.32,0.63,0.92,1.21,1.49,1.77,2.05,2.33,2.61,2.87,3.14,3.40,3.65,3.86,4.04,4.22,4.36,4.47,4.57,4.65,4.69,4.69,4.67,4.68
11,STOCH_RSI_Trend,14,25,48,0.2,0.8,0.8,0.5,9,7,3,25,48,0,410,1.533267,2.907500,218.639333,1192.075000,1.2323,1.1726,1.1443,1.1736,1.1346,1.1825,1.2395,1.2381,1.3347,1.3667,1.4424,1.5659,1.6633,1.6883,1.7016,1.7277,1.7536,1.7613,1.7972,1.8299,1.8879,1.8798,1.9176,1.9630,0.30,0.58,0.87,1.14,1.40,1.66,1.93,2.17,2.42,2.66,2.90,3.13,3.35,3.55,3.72,3.87,4.01,4.12,4.22,4.29,4.34,4.37,4.38,4.40
5,STOCH_RSI_Trend,14,20,48,0.2,0.8,0.8,0.5,9,7,3,15,48,0,140,1.844983,2.905000,118.297667,406.700000,1.4283,1.3068,1.1896,1.2079,1.1673,1.2354,1.3358,1.3838,1.4449,1.5751,1.6984,1.9873,2.1869,2.1707,2.1901,2.2412,2.2808,2.2598,2.2678,2.2780,2.3201,2.3365,2.3833,2.4038,0.27,0.53,0.79,1.05,1.30,1.56,1.82,2.08,2.33,2.58,2.83,3.09,3.32,3.53,3.72,3.89,4.04,4.17,4.30,4.39,4.47,4.52,4.55,4.59
10,STOCH_RSI_Trend,14,25,48,0.2,0.8,0.8,0.5,9,7,3,20,48,0,370,1.523675,2.796250,193.759750,1034.612500,1.2053,1.1411,1.0992,1.1224,1.0833,1.1331,1.1849,1.1985,1.3070,1.3469,1.4132,1.5462,1.6474,1.6791,1.6961,1.7249,1.7592,1.7736,1.8182,1.8556,1.9310,1.9194,1.9649,2.0177,0.28,0.55,0.81,1.07,1.32,1.57,1.83,2.07,2.31,2.55,2.7

# Research parameter influence

In [16]:
param = 'Trend_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Trend_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,3.236211,1.144583,86.068997,44.164444,114
1,3,5.849584,1.552812,62.963132,19.165312,56
2,4,6.061108,1.460000,53.063822,15.588472,32
3,5,6.842036,1.795556,47.689653,14.873611,25
4,6,5.730101,1.770000,37.715247,13.606111,26
5,8,5.441139,1.488750,57.024707,21.980104,61


# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [20],
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod': [48],
                          'low_bound': [0]
                        }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            